In [1]:
%load_ext autoreload
%autoreload 2
%pwd

'/home/tornikeo/Documents/work/scalexa/pangeaai/optimize-cosine'

In [2]:
from cudams.utils import argbatch, mkdir
from cudams.data import get_ref_spectra_from_df
from cudams.kernel import compile
from cudams.utils import name2idx
from cudams.cosine import similarity
import math
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from cudams.data import spectra_peaks_to_tensor
from cudams.processor import Config
from numba import cuda
from itertools import product
from time import perf_counter
from multiprocessing.pool import ThreadPool
from multiprocessing import shared_memory
import numpy as np
import json
import awkward as ak
import matplotlib.pyplot as plt
import matplotlib

assert cuda.is_available()

In [3]:
## Define constants
tolerance: float = 0.1
shift: float = 0
mz_power: float = 0
int_power: float = 1

## How many pairs per batch. Has to be a power of 2.
# Hardware specific - An RTX2070 works best at around 1024 * 2
# But Colab T4 GPU might work best at 1024 * 4
BATCH_SIZE = 2048

# MAX NUMBER OF PEAKS during filtering. Due to nature of matrices, having large number of 
# peaks will increase memory requirements. After 1024, this has diminishing benefits, as 
# smaller and smaller (likely noisy) peaks are taken into consideration when running similarity.
MAX_PEAKS = 1024

# MATCH_LIMIT specifies max how many mz-mz pairs we could consider for each RQ pair, before we sort and filter. 
# E.g. a value of 256 usually causes around ~0.003% of RQ pairs to "overflow".
# The overflown RQ scores will be strictly less than or equal to perfectly accurate score.
# The mean absolute difference at 256, for all overflown pairs is on the order of ~1e-3
# Small values of MATCH_LIMIT (e.g. 128, 64,) cause a dramatic speedup in the processing speed.
MATCH_LIMIT = 1024

# Since Greedy cosine is an unstable algorithm, because approximate mz-mz values do not
# result in approximately the same scores and number of matches.
# So we need to use fp64 to minimize the deviation as much as possible.
# Using float32 causes a significant speedup in the processing speed.
dtype = 'float32'

# Data path
reference_csv_file = Path("data/input/example_dataset_tornike.csv")
query_csv_file = Path("data/input/example_dataset_tornike.csv")

In [4]:
from cudams.processor import CudaCosineGreedy, CpuCosineGreedy
from collections import defaultdict
from matchms import calculate_scores
from matchms.similarity import CosineGreedy
from tqdm import tqdm
from matchms.filtering import normalize_intensities, select_by_mz, select_by_relative_intensity, reduce_to_number_of_peaks, \
    require_minimum_number_of_peaks
from cudams.utils import mute_stdout

def process_spectrum(spectrum: np.ndarray) -> np.ndarray:
    spectrum = select_by_mz(spectrum, mz_from=10.0, mz_to=1000.0)
    spectrum = normalize_intensities(spectrum)
    spectrum = select_by_relative_intensity(spectrum, intensity_from=0.001)
    spectrum = reduce_to_number_of_peaks(spectrum, n_max=MAX_PEAKS)
    spectrum = require_minimum_number_of_peaks(spectrum, n_required=5)
    return spectrum

ref_spectra_df_path = Path(reference_csv_file)
ref_spectra_df = pd.read_csv(ref_spectra_df_path)
references = get_ref_spectra_from_df(ref_spectra_df, spectrum_processor=process_spectrum)

# query_spectra_df_path = Path(query_csv_file)
# query_spectra_df = pd.read_csv(query_spectra_df_path)
# queries = get_ref_spectra_from_df(query_spectra_df, spectrum_processor=process_spectrum)
queries = references[:]


## awkward arrays pip package can handle uneven arrays pretty well. We can use that to
# refs = ak.from_parquet('data/input/example_dataset_tornike.parquet')
# ques = ak.from_parquet('data/input/example_dataset_tornike.parquet')
# data_uneven = ak.from_iter([r.peaks.to_numpy for r in data])
# data_uneven = ak.values_astype(data_uneven, 'float32')
# ak.to_parquet(data_uneven, 'data/input/example_dataset_tornike.parquet')

100%|██████████| 100001/100001 [00:34<00:00, 2877.98it/s]


In [5]:
batches_r = []
for bstart, bend in tqdm(
    argbatch(references, BATCH_SIZE), desc="Batch all references"
):
    rbatch = references[bstart:bend]
    rspec, rlen = spectra_peaks_to_tensor(rbatch, dtype=dtype)
    batches_r.append([rspec, rlen, bstart, bend])


Batch all references: 0it [00:00, ?it/s]

Batch all references: 46it [00:02, 22.98it/s]


In [6]:
batches_q = []
for bstart, bend in tqdm(
    argbatch(queries, BATCH_SIZE), desc="Batch all queries"
):
    qbatch = queries[bstart:bend]
    qspec, qlen = spectra_peaks_to_tensor(qbatch, dtype=dtype)
    batches_q.append([qspec, qlen, bstart, bend])

Batch all queries: 0it [00:00, ?it/s]

Batch all queries: 46it [00:01, 24.50it/s]


In [7]:
batches_rq = list(product(batches_r, batches_q))

TOTAL_BATCHES = len(batches_rq)

batch_outputs = np.empty(shape=(TOTAL_BATCHES,4),dtype=object)
streams = [cuda.stream() for _ in range(TOTAL_BATCHES)]

In [8]:
from cudams.kernel import compile

kernel = compile(
    tolerance=tolerance,
    shift=shift,
    mz_power=mz_power,
    int_power=int_power,
    match_limit=MATCH_LIMIT,
    batch_size=BATCH_SIZE
)

In [9]:

def end_of_stream_callback(
        stream, 
        status, 
        rstart,
        rend,
        qstart,
        qend):
    pass
    # We order a data return
    
    # out = out_cu.copy_to_host(stream=stream)
    # overflow = overflow_cu.copy_to_host(stream=stream)
    # lens = lens_cu.copy_to_host(stream=stream)
    
    # mask = out[:len(rlen),:len(qlen),0] >= threshold
    # # r, c = np.nonzero(mask)
    # out = out[r,c]
    # overflow = overflow[r,c]
    # r += rstart
    # c += qstart
    # batch_outputs[batch_i] = r, c, out, overflow

In [16]:
# We loop over all batchs in sequence
out = np.empty(
    shape=(BATCH_SIZE, BATCH_SIZE, 2),
    dtype="float32",
)

overflow = np.empty(
    shape=(BATCH_SIZE, BATCH_SIZE, 1),
    dtype="uint8",
)
for batch_i in tqdm(range(30)):
    stream = cuda.stream()
    # stream = cuda.default_stream()
    # Each batch has own CUDA stream so that the GPU is as busy as possible

    # We get our batch and lengths (lengths are different for different spectra)
    (rspec, rlen, rstart, rend), (qspec, qlen, qstart, qend) = batches_rq[
        batch_i
    ]
    lens = np.zeros((2, BATCH_SIZE), "int32")
    lens[0, : len(rlen)] = rlen
    lens[1, : len(qlen)] = qlen

    # We make sure main resources remain on CPU RAM
    with cuda.pinned(
        rspec,
        qspec,
        lens,
        out,
        overflow,
    ):
        # We order empty space for results on GPU RAM
        out_cu = cuda.device_array(
            (BATCH_SIZE, BATCH_SIZE, 2), dtype="float32",
            stream=stream
        )
        overflow_cu = cuda.device_array(
            (BATCH_SIZE, BATCH_SIZE, 1), dtype="uint8",
            stream=stream
        )

        # We order the stream to copy input data to GPU RAM
        rspec_cu = cuda.to_device(rspec, stream=stream)
        qspec_cu = cuda.to_device(qspec, stream=stream)
        lens_cu = cuda.to_device(lens, stream=stream)

        # We order the stream to execute kernel (this is scheduled, it will execute, but we can't force it)
        kernel(
            rspec_cu, qspec_cu, lens_cu, out_cu, overflow_cu,
            stream=stream
        )
        # result_output[rstart:rend, qstart:qend] = out
        # result_overflow[rstart:rend, qstart:qend] = overflow
        def cb(stream, status, rstart, rend, qstart, qend):
            print(stream, status, rstart, rend, qstart, qend)
            
        stream.add_callback(
            callback=cb,
            arg=[
                rstart,
                rend,
                qstart,
                qend,
            ],
        )
# We wait for all streams to finish their work everywhere
cuda.synchronize()

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:11<00:00,  2.65it/s]


In [40]:
from cudams.processor import CudaCosineGreedy
from tqdm import tqdm

rlims = argbatch(refs, BATCH_SIZE)
qlims = argbatch(ques, BATCH_SIZE)

R = len(references)
Q = len(queries)

batches_rq = list(product(rlims, qlims))

cosine = CudaCosineGreedy(
    tolerance=tolerance,
    mz_power=0,
    intensity_power=1, 
    shift=0,
    batch_size=BATCH_SIZE,
    match_limit=MATCH_LIMIT,
)
cosine.compile()
t = perf_counter()
ri, qi, out, overflows = cosine.matrix(
    references=references, 
    queries=queries, 
    array_type="sparse",
    sparse_threshold=.75,
)
t = perf_counter() - t
sum_nbytes = sum(o.nbytes for o in [ri, qi, out, overflows])
print(f"Output size {sum_nbytes / 1e9:.2f}GB")
print(f"Num of output {len(ri)}")
print(f"Pairs processed {len(references) * len(queries):.1e}")
n_pairs = len(references) * len(queries)
perh = (n_pairs / t) * 3600

print(f"pairs per hr {perh:.1e}")
print(f"Full run (100kx1.5mln) est: {100_000 * 1_500_000 / perh:.3f}hrs")
print(f"Full run (100kx1.5mln) est GBs: {(sum_nbytes/n_pairs)*(100_000*1_500_000)*1e-9:.2f}GB")

Batch all references: 46it [00:01, 27.33it/s]
Batch all queries: 46it [00:01, 27.86it/s]
  0%|          | 0/2116 [00:00<?, ?it/s]/home/tornikeo/miniconda3/envs/pb2/lib/python3.10/site-packages/numba/cuda/cudadrv/driver.py:2290: UserWarning: Exception in stream callback: CudaCosineGreedy._matrix_with_sparse_output.<locals>.end_of_stream_callback() missing 3 required positional arguments: 'rend', 'qstart', and 'qend'
  warnings.warn(f"Exception in stream callback: {e}")
  3%|▎         | 61/2116 [00:21<14:36,  2.34it/s] Exception ignored in: <finalize object at 0x7f64e452c640; dead>
Traceback (most recent call last):
  File "/home/tornikeo/miniconda3/envs/pb2/lib/python3.10/weakref.py", line 591, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/home/tornikeo/miniconda3/envs/pb2/lib/python3.10/site-packages/numba/cuda/cudadrv/driver.py", line 1660, in core
    driver.cuMemHostUnregister(ptr)
  File "/home/tornikeo/miniconda3/envs/pb2/lib/python3.10/site-packages